# 📘 Co jest "w środku"? - Tomografia komputerowa i uczenie głębokie
---
Ten notebook przedstawia teorię i praktykę tomografii komputerowej (CT) z punktu widzenia przetwarzania sygnałów (transformata Fouriera, Radona, Volume FT) oraz pokazuje, jak zastosować głębokie uczenie (CNN) do analizy obrazów CT.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, iradon
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os, random

# 📘 Analiza i rekonstrukcja tomografii komputerowej (CT) z perspektywy uczenia maszynowego


- [Czym jest tomografia komputerowa (CT)?](#czym-jest-tomografia-komputerowa-ct)
- [Reprezentacja sygnału: transformata Fouriera](#reprezentacja-sygnalu-transformata-fouriera)
- [Volume Fourier Transform – przejście do 3D](#volume-fourier-transform--przejscie-do-3d)
- [Struktura 3D rekonstruowana z wielu 2D projekcji – transformacja Radona i algorytmy odwrotne](#struktura-3d-rekonstruowana-z-wielu-2d-projekcji--transformacja-radona-i-algorytmy-odwrotne)
- [Wizualizacja sinogramu i rekonstrukcji (kod praktyczny)](#wizualizacja-sinogramu-i-rekonstrukcji-kod-praktyczny)
- [Od matematyki do uczenia maszynowego: rola DL w CT](#od-matematyki-do-uczenia-maszynowego-rola-dl-w-ct)
- [Prosty model CNN dla danych CT (praktyka ML)](#prosty-model-cnn-dla-danych-ct-praktyka-ml)
- [Wyniki, obserwacje i dyskusja](#wyniki-obserwacje-i-dyskusja)
- [Podsumowanie i dalsze kierunki rozwoju](#podsumowanie-i-dalsze-kierunki-rozwoju)
- [Bibliografia](#Bibliografia)


## Czym jest tomografia komputerowa (CT)?

Tomografia komputerowa (ang. *Computed Tomography*, w skrócie **CT**) to technika obrazowania,
która pozwala na **rekonstrukcję trójwymiarowej struktury wnętrza obiektu** na podstawie wielu
dwuwymiarowych projekcji (obrazów rentgenowskich) wykonanych pod różnymi kątami.

W przeciwieństwie do klasycznego zdjęcia rentgenowskiego, które nakłada na siebie wszystkie
warstwy ciała, tomografia komputerowa umożliwia uzyskanie **obrazów przekrojowych (ang. slices)**,
czyli oddzielnych warstw wnętrza pacjenta — bez fizycznego rozcinania ciała. ( na szczescie )


---

### ⚙️ Opis matematyczny

Każdy promień w CT można opisać równaniem **Beera–Lamberta**:

$$
I = I_0 \, e^{-\int_L \mu(x, y, z)\, dl}
$$

gdzie:
- $I_0$ — natężenie początkowe promieniowania,
- $I$ — natężenie po przejściu przez ciało,
- $\mu(x, y, z)$ — współczynnik osłabienia promieniowania w danym punkcie,
- $L$ — ścieżka promienia przez ciało.


Pomiar CT rejestruje właśnie te osłabienia — a następnie, przy użyciu **transformacji Radona** i algorytmów odwrotnych, komputer rekonstruuje rozkład $\mu(x,y,z)$, czyli mapę wnętrza.

---

### 📊 Wynik

Wynikiem tomografii komputerowej jest **seria przekrojów 2D** (ang. *slices*), które po zestawieniu dają **model 3D** wnętrza organizmu.
Każdy piksel odpowiada określonemu poziomowi osłabienia promieniowania (w jednostkach **Hounsfielda**, HU):

| Tkanka | Wartość HU (przykład) |
|--------|-----------------------|
| Powietrze | -1000 |
| Płuca | -500 |
| Tkanka miękka | 30–70 |
| Kość | +1000 |


## Reprezentacja sygnału: transformata Fouriera

Transformata Fouriera to jedno z najważniejszych narzędzi w analizie sygnałów.
Pozwala przedstawić sygnał (np. obraz) nie w dziedzinie przestrzeni, ale **w dziedzinie częstotliwości**,
czyli pokazać, z jakich składowych (częstotliwości, fal) jest zbudowany.

Dla obrazu 2D transformata Fouriera ma postać:

$$
F(u, v) = \int\!\!\int f(x, y) \, e^{-j2\pi(ux + vy)} \, dx \, dy
$$

gdzie:
- $f(x,y)$ — obraz w dziedzinie przestrzennej (np. przekrój z tomografii),
- $F(u,v)$ — jego reprezentacja w dziedzinie częstotliwości,
- $(u,v)$ — częstotliwości przestrzenne.

---

### 💡 Dlaczego jest to ważne w tomografii i ML?

- W tomografii komputerowej (CT) każda **projekcja 2D** zawiera informacje o częstotliwościach wnętrza obiektu.
  Dzięki **twierdzeniu projekcyjnemu (Fourier Slice Theorem)** można zrekonstruować pełny obraz z jego widma.

- W uczeniu maszynowym znajomość transformaty Fouriera pomaga zrozumieć:
  - jak sieci konwolucyjne (CNN) reagują na różne częstotliwości w obrazie,
  - dlaczego filtry uczone przez sieć działają jak „lokalne” wersje transformacji Fouriera,
  - jak eliminować szum (filtracja częstotliwościowa) lub poprawiać jakość rekonstrukcji.

---

📘 **W skrócie:**
Transformata Fouriera pokazuje, *z jakich wzorców przestrzennych (częstotliwości)* zbudowany jest obraz —
i stanowi podstawę zarówno dla **rekonstrukcji CT**, jak i **analizy cech w sieciach neuronowych (CNN)**.


## Volume Fourier Transform – przejście do 3D

W tomografii komputerowej celem jest odtworzenie **trójwymiarowego rozkładu osłabienia promieniowania**
wewnątrz obiektu, czyli funkcji $\mu(x, y, z)$.

Aby opisać takie dane w dziedzinie częstotliwości, stosuje się **trójwymiarową transformatę Fouriera** (*Volume Fourier Transform*), zdefiniowaną jako:

$$
F(u, v, w) = \iiint f(x, y, z) \, e^{-j2\pi(ux + vy + wz)} \, dx \, dy \, dz
$$

gdzie:
- $f(x,y,z)$ — funkcja reprezentująca wnętrze obiektu (np. gęstość tkanek),
- $F(u,v,w)$ — jej reprezentacja w dziedzinie częstotliwości,
- $(u,v,w)$ — częstotliwości przestrzenne w trzech wymiarach.

---

### 🧩 Intuicja

Każda **projekcja 2D** (obraz CT pod określonym kątem) odpowiada **płaszczyźnie** w przestrzeni częstotliwości $(u,v,w)$.
Zbierając wiele projekcji pod różnymi kątami, wypełniamy tę przestrzeń i możemy odtworzyć pełną strukturę 3D poprzez **odwrotną transformację Fouriera**.

---

### 💡 Znaczenie w uczeniu maszynowym

- Modele oparte na **3D CNN** lub **U-Net 3D** w zadaniach medycznych uczą się działać w przestrzeni, która odpowiada takiej reprezentacji objętościowej.
- W nowoczesnych podejściach (np. *NeRF*, *Fourier Feature Mapping*) sygnały 3D reprezentuje się również w przestrzeni częstotliwości, aby lepiej modelować zależności przestrzenne.
- W **DL-CT** (deep learning computed tomography) sieć może nauczyć się bezpośrednio odwrotnego odwzorowania z płaszczyzn projekcji do przestrzeni 3D, omijając jawne obliczanie transformacji Fouriera.

---

📘 **W skrócie:**
Transformata Fouriera w 3D (Volume Fourier Transform) opisuje, jak rozkładają się częstotliwości w całej objętości —
i stanowi podstawę matematyczną rekonstrukcji trójwymiarowych struktur w tomografii komputerowej.


## Struktura 3D rekonstruowana z wielu 2D projekcji – transformacja Radona i algorytmy rekonstrukcji odwrotnej

Tomografia komputerowa pozwala odtworzyć **trójwymiarową strukturę wnętrza obiektu** na podstawie wielu
**dwuwymiarowych projekcji (obrazów RTG)** uzyskanych pod różnymi kątami.

Każda taka projekcja rejestruje całkę z funkcji osłabienia $\mu(x, y, z)$ wzdłuż promienia przechodzącego przez obiekt.
Z matematycznego punktu widzenia jest to **transformacja Radona**.

---

### 🧩 Transformacja Radona

Transformacja Radona dla obrazu 2D $f(x,y)$ jest zdefiniowana jako:

$$
p_\theta(s) = \int f(x, y)\,\delta(x\cos\theta + y\sin\theta - s)\,dx\,dy
$$

- $p_\theta(s)$ — projekcja (pomiar CT) pod kątem $\theta$,
- $\delta(\cdot)$ — funkcja Diraca, która „wybiera” punkty leżące na linii projekcji,
- $s$ — pozycja detektora dla danej projekcji.

Zestaw wszystkich projekcji dla różnych kątów $\theta$ tworzy **sinogram** — reprezentację wszystkich danych pomiarowych CT.

---

### 🔄 Rekonstrukcja odwrotna (Inverse Radon Transform)

Aby odzyskać oryginalny obraz $f(x, y)$ (lub strukturę 3D $\mu(x, y, z)$), wykonuje się **odwrotną transformację Radona**.
Najczęściej stosowany algorytm to **Filtered Backprojection (FBP)**:

$$
f(x, y) = \int_0^{\pi} \mathcal{F}^{-1}\{|\omega| P_\theta(\omega)\}\big(x\cos\theta + y\sin\theta\big)\,d\theta
$$

czyli:
1. każda projekcja $p_\theta(s)$ jest filtrowana w dziedzinie Fouriera przez $|\omega|$,
2. następnie „rozsuwana” (backprojection) po wszystkich kątach, tworząc pełen obraz.

---

### 🧠 Intuicja

- Każda **projekcja 2D** wnosi część informacji o wnętrzu obiektu wzdłuż konkretnego kierunku.
- Sumując projekcje z wielu kątów, otrzymujemy kompletny opis przestrzeni — czyli **rekonstrukcję 3D**.
- W praktyce komputer wykonuje setki lub tysiące takich operacji, by uzyskać objętość $f(x, y, z)$.

---

### 💡 Związek z uczeniem głębokim

Nowoczesne modele (np. *FBPConvNet*, *Learned Primal-Dual*, *Neural Radon Inversion*) uczą się **aproksymować transformację odwrotną Radona**,
zastępując klasyczne filtry i całki funkcjami uczonymi przez sieć neuronową.

Dzięki temu można rekonstruować obrazy o wysokiej jakości nawet z **niepełnych danych** (np. z mniejszej liczby projekcji, mniejszą dawką promieniowania).

---

📘 **W skrócie:**
Struktura 3D wnętrza obiektu jest odtwarzana z wielu 2D projekcji dzięki transformacji Radona i algorytmom rekonstrukcji odwrotnej.
To właśnie ten proces pozwala „zajrzeć do środka” bez naruszania obiektu — zarówno w klasycznej tomografii, jak i we współczesnych modelach deep learningowych.


## Bibliografia

- https://www.udemy.com/course/deep-learning-with-pytorch-for-medical-image-analysis/?couponCode=ACCAGE0923




- https://www.youtube.com/watch?v=WxH5bDzMWjg
- https://www.youtube.com/watch?v=f0sxjhGHRPo
- https://www.youtube.com/watch?v=FK4R3waARm8&list=PLldiDnQu2phsW7fsuLW3zmFBSjHItdGXi
- https://www.youtube.com/watch?v=pZ7JlXagT0w
- https://www.youtube.com/watch?v=YIvTpW3IevI
- https://www.youtube.com/watch?v=cSIkpgvio-8
- https://www.youtube.com/watch?v=dn358iX_WxQ






- Principles of Computerized Tomographic Imaging —  (Kak & Slaney) : https://www.mat.uniroma2.it/~picard/SMC/didattica/materiali_did/Anal.Armon./Principles_of_CT_Imaging/Kak&Slaney-CTI-reduced.pdf
- Stabilizing Deep Tomographic Reconstruction : https://arxiv.org/pdf/2008.01846
- DeepLung: Deep 3D Dual Path Nets for
Automated Pulmonary Nodule Detection and Classification : https://arxiv.org/pdf/1801.09555


- honorable mention : Chat GPT
